# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a better model

In [206]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [207]:
datafile_train="../input/edvancer-project-6-health-care/train.csv"
datafile_test="../input/edvancer-project-6-health-care/test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [208]:
medical_hist='../input/edvancer-project-6-health-care/medical_history.csv'
demo_det='../input/edvancer-project-6-health-care/demographic_details.csv'

In [209]:
med=pd.read_csv(medical_hist)

In [210]:
dem=pd.read_csv(demo_det)

In [211]:
# both the datasets have patient id as unique identifier 

In [212]:
med.nunique()

PatientId       62299
Hipertension        2
Diabetes            2
Alcoholism          2
Handcap             5
dtype: int64

In [213]:
dem.nunique()

PatientId        62299
Gender               2
Age                104
Neighbourhood       81
Scholarship          2
dtype: int64

In [214]:
# note that this patient info is for both train and test

In [215]:
comb=pd.merge(med,dem,how='outer',on='PatientId')

In [216]:
comb.head()

,PatientId,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Neighbourhood,Scholarship
0,2.987250e+13,1,0,0,0,F,62,JARDIM DA PENHA,0
1,5.589978e+14,0,0,0,0,M,56,JARDIM DA PENHA,0
2,4.262962e+12,0,0,0,0,F,62,MATA DA PRAIA,0
3,8.679512e+11,0,0,0,0,F,8,PONTAL DE CAMBURI,0
4,8.841186e+12,1,1,0,0,F,56,JARDIM DA PENHA,0


In [217]:
bd_train.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No
1,5.589978e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No
2,8.679512e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No
3,8.841186e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No
4,9.598513e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No


In [218]:
train=pd.merge(bd_train,comb,how='left',on='PatientId')
test=pd.merge(bd_test,comb,how='left',on='PatientId')

In [219]:
train.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Neighbourhood,Scholarship
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No,1,0,0,0,F,62,JARDIM DA PENHA,0
1,5.589978e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No,0,0,0,0,M,56,JARDIM DA PENHA,0
2,8.679512e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No,0,0,0,0,F,8,PONTAL DE CAMBURI,0
3,8.841186e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No,1,1,0,0,F,56,JARDIM DA PENHA,0
4,9.598513e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No,1,0,0,0,F,76,REPÚBLICA,0


In [220]:
# we are going to drop columns 'PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'

# you should try making features out of date columns to improve performance of your model
# try the difference in dates , cyclic features from date components 

In [221]:
drop_cols=['PatientId', 'AppointmentID']

In [222]:
train.drop(drop_cols,1,inplace=True)
test.drop(drop_cols,1,inplace=True)

In [223]:
train['data']='train'
test['data']='test'

all_data=pd.concat([train,test],0,sort=False)

In [224]:
all_data.head()

,ScheduledDay,AppointmentDay,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Neighbourhood,Scholarship,data
0,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No,1,0,0,0,F,62,JARDIM DA PENHA,0,train
1,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No,0,0,0,0,M,56,JARDIM DA PENHA,0,train
2,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No,0,0,0,0,F,8,PONTAL DE CAMBURI,0,train
3,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No,1,1,0,0,F,56,JARDIM DA PENHA,0,train
4,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No,1,0,0,0,F,76,REPÚBLICA,0,train


In [225]:
all_data.isna().sum()

ScheduledDay          0
AppointmentDay        0
SMS_received          0
No-show           10854
Hipertension          0
Diabetes              0
Alcoholism            0
Handcap               0
Gender                0
Age                   0
Neighbourhood         0
Scholarship           0
data                  0
dtype: int64

In [226]:
date_cols = ['ScheduledDay','AppointmentDay']

In [227]:
def get_cyclic(df,cols):
    
    for col in cols : 
        
        df[col]=pd.to_datetime(df[col])
        
        year = str(col+'_year')
        month = str(col+'_month')
        day = str(col+'_day')
        hour = str(col+'hour')
        
        df[day] = df[col].dt.weekday
        df[month] = df[col].dt.month
        df[year] = df[col].dt.year
        df[hour] = df[col].dt.hour
        
        df['day_ref_sin'] = np.sin(2 * np.pi * df[day]/7)
        df['day_ref_cos'] = np.cos(2 * np.pi * df[day]/7)
        
        df['month_ref_sin'] = np.sin(2 * np.pi * df[month]/12)
        df['month_ref_cos'] = np.cos(2 * np.pi * df[month]/12)  
        
        df['hour_ref_sin'] = np.sin(2 * np.pi * df[hour]/24)
        df['hour_ref_cos'] = np.cos(2 * np.pi * df[hour]/24)
        
        del df[col]
        del df[month]
        del df[day]
        del df[hour]

In [228]:
get_cyclic(all_data,date_cols)

In [229]:
all_data.columns

Index(['SMS_received', 'No-show', 'Hipertension', 'Diabetes', 'Alcoholism',
       'Handcap', 'Gender', 'Age', 'Neighbourhood', 'Scholarship', 'data',
       'ScheduledDay_year', 'day_ref_sin', 'day_ref_cos', 'month_ref_sin',
       'month_ref_cos', 'hour_ref_sin', 'hour_ref_cos', 'AppointmentDay_year'],
      dtype='object')

In [230]:
all_data['ScheduledDay_year'] = np.where(all_data['ScheduledDay_year']==2016,1,0)
all_data['AppointmentDay_year'] = np.where(all_data['AppointmentDay_year']==2016,1,0)

In [231]:
all_data['ScheduledDay_year'].value_counts()

1    110280
0        64
Name: ScheduledDay_year, dtype: int64

In [232]:
all_data['gender_f']=(all_data['Gender']=='F').astype(int)
del all_data['Gender']

In [233]:
all_data.shape

(110344, 19)

In [234]:
for col in all_data.columns:
    
    print(col,'\t',all_data[col].dtypes,'\t',all_data[col].nunique())

SMS_received 	 int64 	 2
No-show 	 object 	 2
Hipertension 	 int64 	 2
Diabetes 	 int64 	 2
Alcoholism 	 int64 	 2
Handcap 	 int64 	 5
Age 	 int64 	 104
Neighbourhood 	 object 	 81
Scholarship 	 int64 	 2
data 	 object 	 2
ScheduledDay_year 	 int64 	 2
day_ref_sin 	 float64 	 6
day_ref_cos 	 float64 	 6
month_ref_sin 	 float64 	 3
month_ref_cos 	 float64 	 3
hour_ref_sin 	 float64 	 1
hour_ref_cos 	 float64 	 1
AppointmentDay_year 	 int64 	 1
gender_f 	 int64 	 2


In [235]:
all_data.select_dtypes(include='O').nunique()

No-show           2
Neighbourhood    81
data              2
dtype: int64

In [236]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

all_data['Neighbourhood']=lb.fit_transform(all_data['Neighbourhood'])

In [237]:
scale_cols = ['Age','Neighbourhood']

from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

all_data[scale_cols] = mms.fit_transform(all_data[scale_cols])

In [238]:
all_data.head()

,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Age,Neighbourhood,Scholarship,data,ScheduledDay_year,day_ref_sin,day_ref_cos,month_ref_sin,month_ref_cos,hour_ref_sin,hour_ref_cos,AppointmentDay_year,gender_f
0,0,No,1,0,0,0,0.543103,0.4875,0,train,1,-0.433884,-0.900969,0.866025,-0.5,0.0,1.0,1,1
1,0,No,0,0,0,0,0.491379,0.4875,0,train,1,-0.433884,-0.900969,0.866025,-0.5,0.0,1.0,1,0
2,0,No,0,0,0,0,0.077586,0.6750,0,train,1,-0.433884,-0.900969,0.866025,-0.5,0.0,1.0,1,1
3,0,No,1,1,0,0,0.491379,0.4875,0,train,1,-0.433884,-0.900969,0.866025,-0.5,0.0,1.0,1,1
4,0,No,1,0,0,0,0.663793,0.7250,0,train,1,-0.433884,-0.900969,0.866025,-0.5,0.0,1.0,1,1


In [239]:
target = 'No-show'

In [240]:
X=all_data.drop([target,'data','ScheduledDay_year','hour_ref_sin','hour_ref_cos','AppointmentDay_year'],1)[all_data['data']=='train']
y=all_data[target][all_data['data']=='train']
y = np.where(y=='No',0,1)
x_test=all_data.drop([target,'data','ScheduledDay_year','hour_ref_sin','hour_ref_cos','AppointmentDay_year'],1)[all_data['data']=='test']

In [241]:
y.sum()

20130

In [242]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(x_train,y_train)

pred = lr.predict(x_val)

from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_val,pred)
score

In [243]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

param_dist = {"n_estimators":[100,200,300,500,700,1000],
"max_features": [5,10,20,25,30,35],
"bootstrap": [True, False],
"class_weight":[None,'balanced'],
"criterion":['entropy','gini'],
"max_depth":[None,5,10,15,20,30,50,70],
"min_samples_leaf":[1,2,5,10,15,20],
"min_samples_split":[2,5,10,15,20]}

In [244]:
from sklearn.model_selection import RandomizedSearchCV

n_iter_search = 10

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
n_iter=n_iter_search,
scoring='roc_auc',
cv=5, n_jobs=-1,verbose=False)
random_search.fit(x_train, y_train)

preds = random_search.predict(x_val)

from sklearn.metrics import roc_auc_score

result = roc_auc_score(preds,y_val)
result


0.6990341080591609

In [258]:
submissions=pd.DataFrame({'No-show':random_search.predict_proba(x_test)[:,1]})
submissions.to_csv('project6_Submission3_YogeswaraReddy.csv',index=False)

In [245]:
random_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=15, max_features=5,
                       min_samples_leaf=10, min_samples_split=10,
                       n_estimators=200)

In [246]:
rf = RandomForestClassifier(max_depth=20, max_features=5, min_samples_leaf=15,
                       min_samples_split=5, n_estimators=700)

In [247]:
rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=20, max_features=5, min_samples_leaf=15,
                       min_samples_split=5, n_estimators=700)

In [248]:
feat_imp_df = pd.DataFrame({'features':x_train.columns, 'importance':rf.feature_importances_})
feat_imp_df.sort_values('importance',ascending=False)

,features,importance
5,Age,0.328213
6,Neighbourhood,0.300714
0,SMS_received,0.157409
8,day_ref_sin,0.054569
9,day_ref_cos,0.040339
12,gender_f,0.035089
1,Hipertension,0.016398
11,month_ref_cos,0.015874
10,month_ref_sin,0.015679
7,Scholarship,0.014594


In [255]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost.sklearn import XGBClassifier

In [250]:
xgb_params = {
#"n_estimators":[100,500,700,900,1000,1200,1500],
#"gamma":[0,2,5,8,10],
#"max_depth": [2,3,4,5,6,7,8],
#"min_child_weight":[0.5,1,2,5,10],
#'max_delta_step':[9,10,11,12,13],
#'scale_pos_weight':[1,2,3,4], 
#'subsample':[i/10 for i in range(5,11)],
#'colsample_bytree':[i/10 for i in range(5,11)],
#'colsample_bylevel':[i/10 for i in range(5,11)],
#'reg_lambda':[i/10 for i in range(0,50)],
#'reg_alpha':[i/10 for i in range(0,50)],
#'eta' :[0.01,0.03,0.05,0.07,0.1,0.2,0.3,0.5]
}

In [256]:
xgb_trail = XGBClassifier(n_estimators = 1000,
                          gamma = 5,
                          max_depth = 6,
                          min_child_weight=1,
                          colsample_bylevel=1,
                          colsample_bynode=1,
                          colsample_bytree=1,
                          eval_metric='auc',
                          objective='binary:logistic',
                          scale_pos_weight = 1,
                          subsample = 1,
                          max_delta_step = 9,
                          reg_lambda = 1.7,
                          reg_alpha = 3.8
                          
                          )

#random_search = RandomizedSearchCV(xgb_trail, param_distributions=xgb_params,
#                                   cv=5,scoring = 'roc_auc', verbose=False,n_jobs=-1)

#random_search.fit(x_train,y_train)

xgb_trail.fit(x_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=5, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=9, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=3.8, reg_lambda=1.7, ...)

In [252]:
#print(random_search.best_params_)

In [257]:
#preds = random_search.predict(x_val)
preds = xgb_trail.predict(x_val)

from sklearn.metrics import roc_auc_score

result = roc_auc_score(preds,y_val)
result

0.6094648941887896

In [254]:
sdg

NameError: name 'sdg' is not defined

xgb = XGBClassifier(
    objective="binary:logistic",
    eta = 0.1,
    n_estimators = 500,
    gamma = 7,
    min_child_weight = 5,
    max_depth = 8,
    max_delta_step = 5,
    subsample = 1,
    scale_pos_weight = 1,
    colsample_bytree = 0.7,
    colsample_bylevel = 0.8,
    reg_lambda = 6,
    reg_alpha = 0
)

xgb.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_val,random_search.predict_proba(x_val)[:,1])

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr,tpr,linewidth=2,label=label)
    plt.plot([0,1],[0,1],'k--')

In [ ]:
import matplotlib.pyplot as plt

plot_roc_curve(fpr,tpr)
#plot_roc_curve(y_val,random_search.predict_proba(x_val)[:,1])
plt.show()

In [ ]:
import seaborn as sns

train_score=random_search.predict_proba(x_val)[:,1]
real = y_val

cutoffs = np.linspace(.001,0.999, 999)

KS=[]

for cutoff in cutoffs:
    predicted=(train_score>cutoff).astype(int)
    TP=((real==1)&(predicted==1)).sum()
    FP=((real==0)&(predicted==1)).sum()
    TN=((real==0)&(predicted==0)).sum()
    FN=((real==1)&(predicted==0)).sum()
    ks=(TP/(TP+FN))-(FP/(TN+FP))
    KS.append(ks)

temp=pd.DataFrame({'cutoffs':cutoffs,'KS':KS})
sns.lmplot(x='cutoffs',y='KS',data=temp,fit_reg=False)

In [ ]:
j;;p

In [ ]:
submissions=pd.DataFrame({'No-show':xgb.predict_proba(x_test)[:,1]})
submissions.to_csv('project6_Submission2_YogeswaraReddy.csv',index=False)

In [ ]:
submissions.head()